In [ ]:
import pandas as pd

fake = pd.read_csv("/Users/test/Downloads/archive/Fake.csv")
real = pd.read_csv("/Users/test/Downloads/archive/True.csv")

print("Fake shape:", fake.shape)
print("Real shape:", real.shape)

fake["label"] = 0
real["label"] = 1

df = pd.concat([fake, real], axis=0)
df = df.sample(frac=1).reset_index(drop=True)  # Shuffle

print(df[["text", "label"]].head())
import string
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)  # remove brackets
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = re.sub(f"[{string.punctuation}]", "", text)
    return text

df["clean_text"] = df["text"].apply(clean_text)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

X = df["clean_text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_vec, y_train)

predictions = model.predict(X_test_vec)
print(classification_report(y_test, predictions))


Fake shape: (23481, 4)
Real shape: (21417, 4)
                                                text  label
0  A woman who was born in Nazi Germany says that...      0
1  BEIJING (Reuters) - A Chinese corruption suspe...      1
2  Rep. Maxine Waters is taking a page right out ...      0
3  And we wonder why violence like today s shooti...      0
4  WASHINGTON (Reuters) - Four conservative U.S. ...      1
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      4631
           1       0.92      0.94      0.93      4349

    accuracy                           0.93      8980
   macro avg       0.93      0.93      0.93      8980
weighted avg       0.93      0.93      0.93      8980

